In [ ]:
# !pip install llm=="0.12" chromadb=="0.4.17" watchdog=="3.0.0" langchain langchain_openai supabase pinecone-client
# !llm install llm-clip

In [1]:
import llm
import glob
import base64
import hashlib
import chromadb

In [2]:
from supabase import create_client
from google.colab import userdata

def get_image_url(file_name):
  path = "city_furniture/" + file_name
  url = userdata.get("SUPABASE_URL")
  key = userdata.get("SUPABASE_KEY")
  BUCKET_NAME = "image-bin"

  # Client
  supabase_client = create_client(url, key)
  url = supabase_client.storage.from_(BUCKET_NAME).get_public_url(path)
  return url

In [18]:
# get_image_url("Alden Black Nightstand.jpg")

'https://kxzrpvbbccaxzwlcalcy.supabase.co/storage/v1/object/public/image-bin/city_furniture/Alden Black Nightstand.jpg?'

In [3]:
## Function to use Fetch request
import requests
def fetch(task_id):

  endpoint = "https://api.midjourneyapi.xyz/mj/v2/fetch"

  data = {
      "task_id": task_id
  }

  response = requests.post(endpoint, json=data)

  return response.json()

In [28]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
# from langchain_openai import ChatOpenAI
from google.colab import userdata


## Function to get the summary of the 4 descriptions
def summarize_desc(desc):
  template = """Description: {description}
  Given are 4 different descriptions of a product image. Read through each description to create a single summarized description of the product, the summarized description will be used for a text to image model. Make sure the words are not repeated and use only the above given descriptions to create a summarized description.
  Response: Start the response with the word 'Imagine' """
  api_key = userdata.get("OPEN_API_KEY")
  prompt = PromptTemplate.from_template(template)
  llm = OpenAI(openai_api_key=api_key)
  # llm = ChatOpenAI(temperature=0, openai_api_key=api_key)
  llm_chain = LLMChain(prompt=prompt, llm=llm)

  summarized_description = llm_chain.run(desc)
  return summarized_description

In [16]:
import requests
from google.colab import userdata
import time

### Returns Image description based on Image Url
def get_image_description(img_url):

  X_API_KEY = userdata.get("X_API_KEY")

  endpoint = "https://api.midjourneyapi.xyz/mj/v2/describe"

  headers = {
      "X-API-KEY": X_API_KEY
  }

  data = {
      "image_url": img_url,
      "process_mode": "relax",
  }

  response = requests.post(endpoint, headers=headers, json=data)

  # response.status_code
  desc_output = response.json()
  task_id = desc_output["task_id"]
  time.sleep(30)
  ##### FETCH for description
  fetch_response = fetch(task_id)
  while fetch_response["status"] == "processing":
    time.sleep(10)
    fetch_response = fetch(task_id)
  descriptions = fetch_response["task_result"]["message"]
  summarized_desc = summarize_desc(descriptions)
  return summarized_desc

In [17]:
test_image_desc = get_image_description("https://kxzrpvbbccaxzwlcalcy.supabase.co/storage/v1/object/public/image-bin/city_furniture/Ollie White Nightstand.jpg?")

In [18]:
test_image_desc

'\nImagine a high resolution white nightstand with two drawers and green accents, featuring professional photography and lighting, isolated on a black or white background, in a 16:9 aspect ratio.'

In [12]:
# description_test = """1️⃣ White nightstand with two drawers, front view, white background, green light strip on the right side of the picture, white desk in the back, no shadow, product photography, simple design, modern style, high resolution, high contrast, vibrant colors, natural lighting, professional studio setup, photographed with a Nikon D850 camera and 2470mm f/3.5 lens, studio lighting, in the style of modern product photography. --ar 16:9

# 2️⃣ White nightstand with two drawers, high resolution and realistic image. The white desk has black handles on the front. There is green light coming out from under one edge of its top panel. It stands against an isolated background. High quality photo. --ar 16:9

# 3️⃣ A white wooden nightstand with two drawers and green LED lights, on a white background, with high resolution product photography style in the setting of a modern bedroom. The bottom part has a wood grain texture. It has two large empty spaces for drawing or writing, in the style of a modern bedroom setting. --ar 16:9

# 4️⃣ White nightstand with two drawers, a green neon light in the background, a white wooden texture, high resolution, high quality, detailed, isolated on a black or transparent background. --ar 16:9"""
# summarize_desc(description_test)

'\nImagine a modern white nightstand with two drawers, featuring a sleek design with a white desk in the background and a subtle green light strip on the right side. The high-resolution, high contrast product image is captured with professional studio setup and showcases vibrant colors and natural lighting. Perfect for a modern bedroom, this nightstand also has a wood grain texture and two large empty spaces for drawing or writing. With a green neon light in the background, this white nightstand is a stylish and functional addition to any room. --ar 16:9'

In [19]:
## Generate Embeddings

images = glob.glob("/content/drive/MyDrive/City_Furniture/*.jpg")

model = llm.get_embedding_model("clip")

embeddings = []
for image in images:
  with open(image, "rb") as image_file:
    embedding = model.embed(image_file.read())
    file_name = image.split("/")[-1]
    # TODO: Create a function to get description of the file using the file URL => Description of the Image
    # TODO: Create a function to get summary of the description generated => Summary of desc
    image_url = get_image_url(file_name)
    description = get_image_description(image_url)
    embeddings.append({
      "embedding": embedding,
      "filePath": image,
      "image_url": image_url,
      "description": description,
      "id": base64.b64encode(hashlib.sha256(image.encode()).digest()).decode()
    })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embeddings[10]

In [22]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

# Initialize Pinecone client
pinecone_api_key = userdata.get("PINECONE_API_KEY")
index_name = "test"

pc = Pinecone(api_key=pinecone_api_key)

# Now do stuff
if index_name not in pc.list_indexes().names():
  pc.create_index(
            name=index_name,
            dimension=512,
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )
index = pc.Index(index_name)
# Add embeddings and metadata to the index
for e in embeddings:
    index.upsert(vectors=[
        {
            "id":e["id"],
            "values": e["embedding"],
            "metadata": {
            "filePath": e["filePath"],
            "description": e["description"],
            "image_url": e["image_url"]
                      }
        },
    ])

In [49]:
## Query the embedding

query_embedding = model.embed("A large maplewood bed")
index_response = index.query(
    vector=query_embedding,
    top_k=1,
    includeMetadata= True
)

data = index_response["matches"][0]["metadata"]

In [50]:
data

{'description': '\n\nImagine a white leather bed with a modern, minimalistic design. It features a high gloss finish, silver accents, and a low profile frame. The bed comes in a king size with a high back panel and is adorned with dark blue accent pillows. The product is showcased in high resolution photography on a dark grey background.',
 'filePath': '/content/drive/MyDrive/City_Furniture/Cortina White Uph Platform Bed.jpg',
 'image_url': 'https://kxzrpvbbccaxzwlcalcy.supabase.co/storage/v1/object/public/image-bin/city_furniture/Cortina White Uph Platform Bed.jpg?'}

In [27]:
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 20}},
 'total_vector_count': 20}

In [62]:
import requests
from google.colab import userdata
import time
# Function to generate Image based on the user query

def image_generation(image_url, user_query, description):
  prompt = f"{image_url} {user_query}, {description}"

  X_API_KEY = userdata.get("X_API_KEY")

  endpoint = "https://api.midjourneyapi.xyz/mj/v2/imagine"

  headers = {
      "X-API-KEY": X_API_KEY
  }

  data = {
      "prompt": prompt,
      "process_mode": "fast",
  }

  response = requests.post(endpoint, headers=headers, json=data)
  image_gen_req_output = response.json()
  task_id = image_gen_req_output["task_id"]
  time.sleep(60)

  ##### FETCH for Image
  fetch_response = fetch(task_id)
  while fetch_response["status"] == "processing":
    time.sleep(20)
    fetch_response = fetch(task_id)
  print(fetch_response)
  image = fetch_response["task_result"]["image_url"]
  return image

In [63]:
## Query the embedding

def main():
  query = input("Enter your query: ")
  query_embedding = model.embed(query)
  index_response = index.query(
      vector=query_embedding,
      top_k=1,
      includeMetadata= True
  )

  data = index_response["matches"][0]["metadata"]

  image_url = data["image_url"].replace(" ","%20")
  description = data["description"]

  url = image_generation(image_url, query, description)
  return url

In [65]:
main()

Enter your query: A Large Bench
{'task_id': 'fe035e06-015c-40e4-aa14-eca27dabd3b3', 'status': 'finished', 'process_time': 46, 'retry_count': 0, 'meta': {'account_id': '2858', 'task_type': 'imagine', 'origin_task_id': '', 'bot_hash': '', 'bot_id': 0, 'model_version': 'unknown', 'process_mode': 'fast', 'quota_frozen': 14, 'frozen_credit': 0, 'created_at': 1712848215, 'created_at_utc': '2024-04-11 15:10:15 +0000 UTC', 'started_at': 1712848233, 'started_at_utc': '2024-04-11 15:10:33 +0000 UTC', 'ended_at': 1712848279, 'ended_at_utc': '2024-04-11 15:11:19 +0000 UTC', 'task_request': {'aspect_ratio': '', 'auto_upload': False, 'bot_id': 0, 'process_mode': 'fast', 'prompt': 'https://kxzrpvbbccaxzwlcalcy.supabase.co/storage/v1/object/public/image-bin/city_furniture/Logan%20Dark%20Tone%20Nightstand.jpg? A Large Bench, Imagine a product with four different descriptions - a unique, all-in-one solution for your needs.', 'skip_prompt_check': False, 'skip_retry': False, 'upload_prompt_image': False},

'https://img.midjourneyapi.xyz/mj/fe035e06-015c-40e4-aa14-eca27dabd3b3.png'

## Test Results


<h4>Query: A black nightstand</h4>
<img src="https://img.midjourneyapi.xyz/mj/07161110-5ce3-4ff3-b407-db5241719523.png">


<h4>Query: A large maplewood bed </h4>
<img src="https://img.midjourneyapi.xyz/mj/a6cf2a63-216b-466c-a17a-4f09a432a0bb.png">

<h4>Query: A large bench </h4>
<img src="https://img.midjourneyapi.xyz/mj/fe035e06-015c-40e4-aa14-eca27dabd3b3.png">